## Exercise: Write Allocates

**Ideally, this example should be run on Noctua 1.**

In [2]:
function striad!(a, b, c, d)
    for i in eachindex(a, b, c, d)
        a[i] = b[i] + c[i] * d[i]
    end
    return nothing
end

N = 1_000_000
a = rand(N)
b = rand(N)
c = rand(N)
d = rand(N)

striad!(a, b, c, d)

1) Looking at the Schoenhauer Triad kernel (i.e. the `striad!` function above),
how many LOADs (data reads) and STOREs (data writes) to you expect to happen? Otherwise put, how many bytes do
you think will need to be transferred to/from memory?

2) Use LIKWID.jl to empirically measure how much data has been read from / written to memory.
  - Hint: Depending on availability, you want to measure the "DATA" or "MEM" performance group.

In [3]:
using LIKWID

In [4]:
# ... Your code goes here ...
@perfmon "DATA" striad!(a, b, c, d);


Group: DATA
┌─────────────────────────────┬───────────┐
│                       Event │  Thread 1 │
├─────────────────────────────┼───────────┤
│           INSTR_RETIRED_ANY │ 1.19004e6 │
│       CPU_CLK_UNHALTED_CORE │ 3.43582e6 │
│        CPU_CLK_UNHALTED_REF │ 8.11652e6 │
│  MEM_INST_RETIRED_ALL_LOADS │  751102.0 │
│ MEM_INST_RETIRED_ALL_STORES │  250483.0 │
└─────────────────────────────┴───────────┘
┌──────────────────────┬────────────┐
│               Metric │   Thread 1 │
├──────────────────────┼────────────┤
│  Runtime (RDTSC) [s] │ 0.00318127 │
│ Runtime unhalted [s] │ 0.00132557 │
│          Clock [MHz] │     1097.2 │
│                  CPI │    2.88713 │
│  Load to store ratio │    2.99861 │
└──────────────────────┴────────────┘


3) Which ratio of reads and writes do you find? How many LOADs and STOREs actually happen per iteration?

The reason you might see a higher load/store ratio are so-called "write-allocates": On some systems, to write to a piece of memory it has to be loaded to cache first (e.g. by reading from it first). Hence you get one extra LOAD.

4) In the exercise "cache_sizes" we used SDAXPY rather than STRIAD.
  * How would the bandwidth values for striad (qualitatively) compare to our
    sdaxpy results assuming we didn't account for write-allocates?
  * Focusing on data volume rather than data transfer,
    how much data is hold for one iteration of sdaxpy and striad, respectively?
    Does a factor of this data volume fit nicely into L1 cache (in either case)?